In [1]:
import os 
import sys 
import logging 
from functools import cache 

logging.basicConfig(level=logging.WARNING)

cur_path = os.path.abspath("../..")
if cur_path not in sys.path: 
    sys.path.append(cur_path)

import numpy as np 
import pandas as pd 
import altair as alt 
from palettable.scientific.sequential import Batlow_5
from altair import datum
from dotenv import load_dotenv
from subgrounds.subgrounds import Subgrounds, Subgraph
from subgrounds.subgraph import SyntheticField
from subgrounds.pagination import ShallowStrategy

# Required when developing in a jupyter-notebook environment 
load_dotenv('../../../../.env')

from utils_notebook.utils import remove_prefix, ddf, load_subgraph
from utils_notebook.vega import (
    output_chart, 
    wide_to_longwide,  
    chart,
    apply_css
) 
from utils_notebook.css import css_tooltip_timeseries_multi_colored
from utils_notebook.queries import QueryManager
from utils_notebook.testing import validate_season_series

In [2]:
sg, bs = load_subgraph()
q = QueryManager(sg, bs)

In [3]:
df = q.query_barn()
df = df.iloc[1:,] # TODO: figure out why this is here 
df = df.resample("D", on="timestamp").last().reset_index()
df.head()

timestamp  season  reward_fertilized_sprouts  sprouts_rinsable  \
0 2022-08-06    6081               16998.036930      5.349974e+04   
1 2022-08-07    6105               56197.990557      1.912626e+06   
2 2022-08-08    6129                   0.000000      1.996040e+06   
3 2022-08-09    6153                   0.000000      1.998324e+06   
4 2022-08-10    6177                   0.000000      2.031036e+06   

        sprouts  rinsable_percent  fertilizer_total  fertilizer_active  \
0  8.666069e+07          0.000617          77000000         14515783.0   
1  9.354104e+07          0.020037          77000000         17040568.0   
2  9.390093e+07          0.020814          77000000         17169557.0   
3  9.396048e+07          0.020825          77000000         17188268.0   
4  9.395637e+07          0.021159          77000000         17197370.0   

   fertilizer_available  
0            62484217.0  
1            59959432.0  
2            59830443.0  
3            59811732.0  
4            59802630.0

In [4]:
# sprouts breakdown 
df_sprouts = wide_to_longwide(
    df, 
    "timestamp", 
    ['season', 'timestamp'], 
    ['sprouts_rinsable', 'sprouts', 'rinsable_percent'],
    sort_col="timestamp",
)
colors = {
    'sprouts_rinsable': Batlow_5.hex_colors[2], 
    'sprouts': Batlow_5.hex_colors[0], 
    'rinsable_percent': Batlow_5.hex_colors[3], 
}
chart_sprouts_breakdown = chart(
    df_sprouts, 
    "timestamp", 
    lmetrics=['sprouts', 'sprouts_rinsable'], 
    rmetrics=['rinsable_percent'], 
    lstrategy="stack_area", 
    rstrategy="line", 
    title="Sprouts Breakdown",
    dual_axes=True, 
    show_exploit_rule=False, 
    xaxis_kwargs=dict(labelExpr="timeFormat(toDate(datum.value), '%b %e, %Y')"),
    yaxis_left_kwargs=dict(title="Sprouts", format=".3~s"), 
    yaxis_right_kwargs=dict(title="Percent", format=".2%"), 
    tooltip_formats={
        'sprouts_rinsable': ',d', 
        'sprouts': ',d', 
        'rinsable_percent': '.2%'
    }, 
    colors=colors, 
    width=400, 
)

css_lines = css_tooltip_timeseries_multi_colored(list(colors.keys()), colors) 
css = "\n".join(css_lines)

apply_css("")
# apply_css(css)

chart_sprouts_breakdown

/Users/ALEX/opt/anaconda3/envs/beanstalk/lib/python3.10/site-packages/altair/utils/core.py:317: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for col_name, dtype in df.dtypes.iteritems():


alt.LayerChart(...)

In [5]:
# fertilizer breakdown 
series_fert = df.iloc[-1,][['fertilizer_available', 'fertilizer_active']]
fert_total = series_fert['fertilizer_available'] + series_fert['fertilizer_active']
df_fert = pd.DataFrame(
    {
        'category': series_fert.index, 
        'value': series_fert.values, 
        'label': [
            f"{c}: {int(v):,} \n({v / fert_total:.1%})" 
            for c, v in zip(series_fert.index, series_fert.values)
        ]
    }
)

chart_fert_breakdown = (
    alt.Chart(df_fert, title="Fertilizer Breakdown")
    .encode(
        theta=alt.Theta("value:Q", stack=True),
        color=alt.Color(
            "category:N", 
            legend=alt.Legend(), 
            scale=alt.Scale(
                domain=['fertilizer_available', 'fertilizer_active'], 
                range=[Batlow_5.hex_colors[4], Batlow_5.hex_colors[1]]
            )
        ),
        tooltip="label:N"
    )
    .mark_arc(outerRadius=120, innerRadius=75)
)

c = (
    alt.hconcat(chart_sprouts_breakdown, chart_fert_breakdown)
    .resolve_legend(color="independent")
    .resolve_scale(y="independent", color="independent")
    .resolve_axis(y="independent")
)
c

/Users/ALEX/opt/anaconda3/envs/beanstalk/lib/python3.10/site-packages/altair/utils/core.py:317: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for col_name, dtype in df.dtypes.iteritems():


alt.HConcatChart(...)

In [7]:
df_fert.head()

category       value  \
0  fertilizer_available  59784667.0   
1     fertilizer_active  17215333.0   

                                        label  
0  fertilizer_available: 59,784,667 \n(77.6%)  
1     fertilizer_active: 17,215,333 \n(22.4%)

In [ ]:
# TODO: update css for this chart 
output_chart(c, css=css)